In [1]:
import random
import json

import numpy as np

import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

import math

In [2]:
class WordleCritic:
    def __init__(self, words_file):
        with open(words_file + ".json", "r") as f:
            words = json.load(f)
        
        self.word = random.choice(words)
        
    def judge(self, suggestion):
        result = ["gray" for i in range(len(self.word))]
        
        for index, letter in enumerate(suggestion):
            if letter == self.word[index]:
                result[index] = "green"
            elif letter in self.word:
                result[index] = "yellow"
                
        return result

In [3]:
class WordlePlayer:
    def __init__(self, words_file):
        self.words_pd = pd.read_csv(words_file + ".csv")
        
        with open(words_file + ".json", "r") as f:
            self.words_list = json.load(f) 
        
        self.model = BayesianNetwork([
            ("first", "second"), 
            ("third", "second"), 
            ("third", "forth"), 
            ("fifth", "forth")
        ])

        self.model.fit(self.words_pd, estimator=MaximumLikelihoodEstimator)
        self.infer = VariableElimination(self.model)
        
    def get_suggestion_word(self, suggestion, evidence):
        word = ["", "", "", "", ""]

        if "first" in evidence:
            word[0] = evidence["first"]
        else: 
            word[0] = suggestion["first"]

        if "second" in evidence:
            word[1] = evidence["second"]
        else:
            word[1] = suggestion["second"]

        if "third" in evidence:
            word[2] = evidence["third"]
        else:
            word[2] = suggestion["third"]

        if "forth" in evidence:
            word[3] = evidence["forth"]
        else:
            word[3] = suggestion["forth"]

        if "fifth" in evidence:
            word[4] = evidence["fifth"]
        else:
            word[4] = suggestion["fifth"]

        return word

    def word_is_valid(self, word, must_contain=[], must_not_contain=[], must_not_contain_at={}):
        if "".join(str(char) for char in word) in self.words_list:

            for letter in must_contain:
                if not letter in word:
                    return False

            for letter in must_not_contain:
                if letter in word:
                    return False

            if "first" in must_not_contain_at and word[0] in must_not_contain_at["first"]:
                return False
            
            if "second" in must_not_contain_at and word[1] in must_not_contain_at["second"]:
                return False

            if "third" in must_not_contain_at and word[2] in must_not_contain_at["third"]:
                return False

            if "forth" in must_not_contain_at and word[3] in must_not_contain_at["forth"]:
                return False

            if "fifth" in must_not_contain_at and word[4] in must_not_contain_at["fifth"]:
                return False

            return True
        else:
            return False

    def get_suggestion(self, variables, evidence, must_contain=[], must_not_contain=[], must_not_contain_at=[]):
        q = self.infer.query(variables, evidence=evidence, show_progress=False)

        count_predictions = len(q.values.flatten()[q.values.flatten() != 0])
        max_value_indices = (-q.values.flatten()).argsort()[:count_predictions]

        result = []

        for max_value_index in max_value_indices:
            indices = np.unravel_index(max_value_index, q.values.shape)

            suggestion = {}

            for index, variable in enumerate(q.variables):
                suggestion[variable] = self.model.get_cpds(variable).state_names[variable][indices[index]]

            word = self.get_suggestion_word(suggestion, evidence)

            if self.word_is_valid(word, must_contain, must_not_contain, must_not_contain_at):
                #result.append(word)
                return word

        return result
    
    def get_first_suggestion(self):
        first = self.infer.map_query(["first"], show_progress=False)["first"]
        second = self.infer.map_query(["second"], evidence={"first": first}, show_progress=False)["second"]
        third = self.infer.map_query(["third"], evidence={"first": first, "second": second}, show_progress=False)["third"]
        forth = self.infer.map_query(["forth"], evidence={"first": first, "second": second, "third": third}, show_progress=False)["forth"]
        fifth = self.infer.map_query(["fifth"], evidence={"first": first, "second": second, "third": third, "forth": forth}, show_progress=False)["fifth"]

        if self.word_is_valid("".join([first,second,third,forth,fifth])):
            return "".join[first,second,third,forth,fifth]
        elif len(self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third) & 
                (self.words_pd["forth"] == forth)]) > 0:
            return self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third) & 
                (self.words_pd["forth"] == forth)].values[0]
        else:
            return self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third)].values[0]
        

In [6]:
critic = WordleCritic("wordle-at-words")

print(critic.word)

FLUSE


In [7]:
player = WordlePlayer("wordle-at-words")

suggestion = player.get_first_suggestion()

new_input = {
    "variables":set([]), 
    "evidence":{}, 
    "must_contain":set([]), 
    "must_not_contain":set([]),
    "must_not_contain_at":{
        "first":set([]), 
        "second":set([]), 
        "third":set([]),
        "forth":set([]),
        "fifth":set([])
    }
}

tries = 0

while True:
    evidences = critic.judge(suggestion)
    tries += 1
    
    print(str(tries) + ": " + "".join(suggestion))
    print(evidences)
    print()
    
    if evidences[0] == "green" and evidences[1] == "green" and evidences[2] == "green" and evidences[3] == "green" and evidences[4] == "green":
        break
    
    if evidences[0] == "gray":
        new_input["variables"].add("first")
        new_input["must_not_contain"].add(suggestion[0])
    elif evidences[0] == "yellow":
        new_input["variables"].add("first")
        new_input["must_contain"].add(suggestion[0])
        new_input["must_not_contain_at"]["first"].add(suggestion[0])
    elif evidences[0] == "green":
        new_input["variables"].discard("first")
        new_input["evidence"]["first"] = suggestion[0]

    if evidences[1] == "gray":
        new_input["variables"].add("second")
        new_input["must_not_contain"].add(suggestion[1])
    elif evidences[1] == "yellow":
        new_input["variables"].add("second")
        new_input["must_contain"].add(suggestion[1])
        new_input["must_not_contain_at"]["second"].add(suggestion[1])
    elif evidences[1] == "green":
        new_input["variables"].discard("second")
        new_input["evidence"]["second"] = suggestion[1]

    if evidences[2] == "gray":
        new_input["variables"].add("third")
        new_input["must_not_contain"].add(suggestion[2])
    elif evidences[2] == "yellow":
        new_input["variables"].add("third")
        new_input["must_contain"].add(suggestion[2])
        new_input["must_not_contain_at"]["third"].add(suggestion[2])
    elif evidences[2] == "green":
        new_input["variables"].discard("third")
        new_input["evidence"]["third"] = suggestion[2]

    if evidences[3] == "gray":
        new_input["variables"].add("forth")
        new_input["must_not_contain"].add(suggestion[3])
    elif evidences[3] == "yellow":
        new_input["variables"].add("forth")
        new_input["must_contain"].add(suggestion[3])
        new_input["must_not_contain_at"]["forth"].add(suggestion[3])
    elif evidences[3] == "green":
        new_input["variables"].discard("forth")
        new_input["evidence"]["forth"] = suggestion[3]

    if evidences[4] == "gray":
        new_input["variables"].add("fifth")
        new_input["must_not_contain"].add(suggestion[4])
    elif evidences[4] == "yellow":
        new_input["variables"].add("fifth")
        new_input["must_contain"].add(suggestion[4])
        new_input["must_not_contain_at"]["fifth"].add(suggestion[4])
    elif evidences[4] == "green":
        new_input["variables"].discard("fifth")
        new_input["evidence"]["fifth"] = suggestion[4]

    suggestion = player.get_suggestion(
        new_input["variables"], 
        new_input["evidence"],
        new_input["must_contain"],
        new_input["must_not_contain"],
        new_input["must_not_contain_at"]
    )
    
print("The word you are looking for is " + "".join(suggestion) + "!")
print("Found it in " + str(tries) + " tries!")

1: SABOR
['yellow', 'gray', 'gray', 'gray', 'gray']

2: LEINS
['yellow', 'yellow', 'gray', 'gray', 'yellow']

3: FLUSE
['green', 'green', 'green', 'green', 'green']

The word you are looking for is FLUSE!
Found it in 3 tries!
